<a href="https://colab.research.google.com/github/saidnaqwe/Python-Project/blob/main/MachineLearningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/negojovanovich3/pythonprojects/main/diabetic_data.csv")

In [ ]:
data = data.dropna()

In [ ]:
#eda
readmit = data[data["readmitted"]!="NO"]
No_readmit = data[data["readmitted"] == "NO"]

sns.countplot(data["readmitted"])
plt.figure(figsize=(10,5))

sns.countplot(x ="race", data=readmit, hue ="age")
plt.legend(loc ="upper right",title="age of patient")

plt.figure(figsize=(10,5))

sns.countplot(x ="race", data=No_readmit, hue ="age")
plt.legend(loc ="upper right",title="age of patient")

In [ ]:
#data cleaning
df_knn =data.drop(data.columns[10:31], axis=1)
df_knn

#tyrty

In [ ]:
#drop columns and create dummy columns for each remaining
df_knn.drop(['race','gender','age','weight','encounter_id','change','diabetesMed','examide','citoglipton','encounter_id','patient_nbr','readmitted'], axis=1, inplace=True)
df_knn= pd.get_dummies(data=df_knn, columns=['glyburide','tolbutamide','pioglitazone',	'rosiglitazone',	'acarbose',	'miglitol'	,'troglitazone',
                                             'tolazamide'	,'insulin','glyburide-metformin',	'glipizide-metformin'	,
                                             'glimepiride-pioglitazone',	'metformin-rosiglitazone','metformin-pioglitazone',
                                             	])

In [ ]:
df_knn = df_knn.drop(['admission_source_id','discharge_disposition_id','admission_type_id'], axis=1)

In [ ]:
df_knn

In [ ]:
#Create a correlation matrix that measures the linear relationships between the variables
correlation_matrix = df_knn.corr().round(1)
plt.figure(figsize =(25,15))
sns.heatmap(data=correlation_matrix, annot=True, fmt=".2g", square = True, )


In [ ]:
x_features = df_knn
y_target = df_knn["time_in_hospital"]



In [ ]:
#linear model
from sklearn.linear_model import LinearRegression

lineReg = LinearRegression()
lineReg.fit(x_features,y_target)

In [ ]:
print('the estimated intercept %.2f '%lineReg.intercept_)
print('the coefficient is %d ' %len(lineReg.coef_))



In [ ]:
#MODEL TRAINING
# train model split the whole dataset into train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_features,y_target)

print(x_features.shape)

lineReg.fit(X_train,Y_train)



In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# model evaluation for testing set
y_train_predict = lineReg.predict(X_train)

# root mean square error of the model
rmse = np.sqrt(mean_squared_error(Y_train,y_train_predict))

# r-squared score of the model
r2 = r2_score(Y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

In [ ]:
# plotting the y_test vs y_pred
# ideally should have been a straight line
plt.scatter(Y_test, y_test_predict)
plt.show()

In [ ]:

#data cleaning
df_data =data.drop(["encounter_id","patient_nbr",],axis =1)

df_data =df_data.drop(data.columns[15:48], axis=1)

#for race
race_dummies = pd.get_dummies(df_data['race'],)
df_data.drop(['race'], axis=1, inplace=True)
df_data_new = pd.concat([df_data,race_dummies],axis=1)


#for gender
gender_dummies = pd.get_dummies(df_data['gender'],)
df_data_new.drop(['gender'], axis=1, inplace=True)
df_data_new = pd.concat([df_data_new,gender_dummies],axis=1)

#for age
#age_dummies = pd.get_dummies(df_data['age'],)
#df_data_new.drop(['age'], axis=1, inplace=True)
#df_data_new= pd.concat([df_data_new,age_dummies],axis=1)

#for diabetes
#diaMeds_dummies = pd.get_dummies(df_data['diabetesMed'],prefix="diabetesMed")
#df_data_new.drop(['diabetesMed'], axis=1, inplace=True)
#df_data_new = pd.concat([df_data_new,diaMeds_dummies],axis=1)

# for readmit
readmit_dummies = pd.get_dummies(df_data['readmitted'],prefix ="readmitted" )
df_data_new.drop(['readmitted'], axis=1, inplace=True)
df_data_new = pd.concat([df_data_new,readmit_dummies],axis=1)

df_data_new = df_data_new.drop(["diabetesMed","age","Unknown/Invalid","weight","admission_type_id","discharge_disposition_id","admission_source_id","payer_code", "medical_specialty"], axis=1)
df_2 = df_data_new.drop(['?', "Other"], axis =1)


In [ ]:

df_2


In [ ]:
#standardizes the data by subtracting the mean and then scaling to unit variance.(Unit variance means dividing all the values by the standard deviation)
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()

scalar.fit(df_2.drop('Hispanic', axis=1))
scaled_features =scalar.transform(df_2.drop('Hispanic', axis =1))
df_kmm_features = pd.DataFrame(scaled_features, columns = df_2.columns[:-1])



In [ ]:
#split data
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2  = train_test_split(scaled_features, df_2['Hispanic'], test_size =.3)

# gathering data for KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train2,y_train2)
predict2 = knn.predict(X_test2)
predict2



In [ ]:
# matrix compares the actual target values with those predicted by the machine learning model.
from sklearn.metrics import  classification_report, confusion_matrix
print(confusion_matrix(y_test2,predict2))



In [ ]:
#displaying confusin matrix
plt.figure(figsize=(10,5))
sns.heatmap(confusion_matrix(y_test2, predict2), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'viridis');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(knn.score(X_test2, y_test2))
plt.title(all_sample_title, size = 15);



In [ ]:
print(classification_report(y_test2,predict2))

#module implements several loss, score, and utility functions to measure classification performance
from sklearn import metrics

score =[]

for i in range(1,40):
  knn2 = KNeighborsClassifier(n_neighbors=i)
  knn2.fit(X_train2, y_train2)
  y_pred = knn2.predict(X_test2)
  score.append(metrics.accuracy_score(y_test2,y_pred))

print(score)



In [ ]:
#plotting accuracy score with the k value
plt.figure(figsize=(10,6))
plt.plot(range(1,40),score, color='blue')
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')



In [ ]:
# calculate the mean of error for all the predicted values where K ranges from 1 and 40.
#executes a loop from 1 to 40. In each iteration the mean error for predicted values of test set is calculated and the result is appended to the error list.
error_rate = []
for i in range(1,40):
    
    knn3 = KNeighborsClassifier(n_neighbors=i)
    knn3.fit(X_train2,y_train2)
    pred_i = knn3.predict(X_test2)
    error_rate.append(np.mean(pred_i != y_test2))



#plotting error vate to k value

plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')